In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import zipfile
import os
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Tentukan path dataset
dataset_path = '/content/drive/MyDrive/archive.zip'

# Verifikasi file
!ls $dataset_path

/content/drive/MyDrive/archive.zip


In [7]:
zip_file = r"/content/drive/MyDrive/archive - Copy.zip"
# Ekstrak file ZIP
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# Verifikasi isi folder hasil ekstraksi
!ls /content/dataset
!ls /content/dataset/DATASET


DATASET
TEST  TRAIN  VAL


In [8]:
test_dir = '/content/dataset/DATASET/TEST'
train_dir = '/content/dataset/DATASET/TRAIN'
val_dir = '/content/dataset/DATASET/VAL'


In [18]:
# Buat dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(128, 128),
    batch_size=32
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(128, 128),
    batch_size=32
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(128, 128),
    batch_size=32
)



Found 16011 files belonging to 3 classes.
Found 3993 files belonging to 3 classes.
Found 3062 files belonging to 3 classes.


In [10]:
# Ambil nama kelas untuk setiap dataset
train_class_names = train_dataset.class_names
val_class_names = val_dataset.class_names
test_class_names = test_dataset.class_names

print("Kelas pada dataset TRAIN:", train_class_names)
print("Kelas pada dataset VAL:", val_class_names)
print("Kelas pada dataset TEST:", test_class_names)


Kelas pada dataset TRAIN: ['anorganik', 'b3', 'organik']
Kelas pada dataset VAL: ['anorganik', 'b3', 'organik']
Kelas pada dataset TEST: ['anorganik', 'b3', 'organik']


In [11]:
# Prefetch dataset untuk efisiensi
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [12]:
data_augmentation = Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

In [13]:
# augmentasi
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)


In [20]:
# Model CNN
model = Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_class_names), activation='softmax')
])

# Kompilasi model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Latih model dan simpan hasilnya dalam history
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.6560 - loss: 0.7818 - val_accuracy: 0.7493 - val_loss: 0.5900
Epoch 2/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 15s 29ms/step - accuracy: 0.7533 - loss: 0.6156 - val_accuracy: 0.7944 - val_loss: 0.5313
Epoch 3/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.7812 - loss: 0.5602 - val_accuracy: 0.7914 - val_loss: 0.5251
Epoch 4/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.7983 - loss: 0.5162 - val_accuracy: 0.8012 - val_loss: 0.4999
Epoch 5/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - accuracy: 0.8106 - loss: 0.4824 - val_accuracy: 0.8197 - val_loss: 0.4668
Epoch 6/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.8274 - loss: 0.4494 - val_accuracy: 0.8307 - val_loss: 0.4430
Epoch 7/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 15s 30ms/step - accuracy: 0.8354 - loss: 0.4305 - val_accuracy: 0.8272 - val_loss: 0.4495
Epoch 8/10
501/501 ━━━━━━━━━━━━━━━━━━━━ 15s 30ms/step - accuracy: 0.8445 - loss: 0.4073 - 

In [24]:
# Evaluasi model pada dataset validasi
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f"Akurasi pada dataset validasi: {val_accuracy:.2f}")


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8377 - loss: 0.4255
Akurasi pada dataset validasi: 0.84
